In [81]:
import librosa
import librosa.display
import os
import numpy as np
from scipy.signal import get_window
from sklearn import preprocessing
import python_speech_features as mfcc
from sklearn.mixture import GaussianMixture 
import pickle


## Feartures Extraction

##### MFCC Feature

In [82]:
def calculate_delta(array):
	
    rows,cols = array.shape
    print(rows)
    print(cols)
    deltas = np.zeros((rows,20))
    N = 2
    for i in range(rows):
        index = []
        j = 1
        while j <= N:
            if i-j < 0:
              first =0
            else:
             first = i-j
            if i+j > rows-1:
                second = rows-1
            else:
                second = i+j 
            index.append((second,first))
            j+=1
        deltas[i] = ( array[index[0][0]]-array[index[0][1]] + (2 * (array[index[1][0]]-array[index[1][1]])) ) / 10
    return deltas


def extract_features(file_path):
    audio , sample_rate = librosa.load(file_path, res_type='kaiser_fast')
    mfcc_feature = mfcc.mfcc(audio,sample_rate, 0.025, 0.01,20,nfft = 1200, appendEnergy = True)    
    mfcc_feature = preprocessing.scale(mfcc_feature)
    print(mfcc_feature)
    delta = calculate_delta(mfcc_feature)
    combined = np.hstack((mfcc_feature,delta)) 
    return combined

In [83]:



featuresMagdy = []
directory = './files/magdy/'
for audio in os.listdir('./files/magdy/'):
    audio_path = directory + audio
    featuresMagdy=np.vstack(extract_features(audio_path) )



featuresMayar = []
directory = './files/mayar/'
for audio in os.listdir('./files/mayar/'):
    audio_path = directory + audio
    featuresMayar=np.vstack(extract_features(audio_path) )

featuresMina = []
directory = './files/Mina/'
for audio in os.listdir('./files/Mina/'):
    audio_path = directory + audio
    featuresMina=np.vstack(extract_features(audio_path) )

featuresMostafa = []
directory = './files/mostafa/'
for audio in os.listdir('./files/mostafa/'):
    audio_path = directory + audio
    featuresmostafa=np.vstack(extract_features(audio_path) )

[[-7.03832194 -0.97406876  0.87284937 ...  0.0441629   0.45870883
  -0.43213518]
 [-0.11565193 -2.41559971 -2.00529497 ... -0.36040543  1.41108939
  -0.79848436]
 [ 0.02140605 -2.38487934 -1.98169001 ... -0.34463678  1.35300335
  -0.81815959]
 ...
 [ 0.447785    1.38718736  1.14321559 ...  1.49612855  0.02208254
  -0.34717586]
 [ 0.44717706  0.82610567  1.05673206 ...  0.68754956  1.06505683
  -0.12715405]
 [ 0.43459773 -0.30403371  1.28251999 ... -0.3503555   1.18093711
   0.42403737]]
284
20
[[-6.67361414  0.06756381  0.43491561 ...  0.21900015  0.57009275
  -0.14267845]
 [-0.64666703 -1.17980882  1.94238162 ... -1.13715907  0.80120436
   0.89107757]
 [-0.45261108 -1.17289357  1.74325451 ... -0.71696136 -0.489671
   0.63686166]
 ...
 [-0.78518542 -1.11565827 -1.12755487 ... -1.8716987   0.71071738
   1.17560369]
 [-0.8052587  -1.1460631  -1.2943425  ... -1.31922721  1.28181512
   1.49412935]
 [-0.91168121 -1.18114455 -1.33013472 ... -1.03390012  1.20941065
   0.84892632]]
293
20
[[-4

In [84]:



magdy_gmm = GaussianMixture(n_components = 6, max_iter = 2000, covariance_type='diag',n_init = 3)
magdy_gmm.fit(featuresMagdy)


mayar_gmm = GaussianMixture(n_components = 6, max_iter = 2000, covariance_type='diag',n_init = 3)
mayar_gmm.fit(featuresMayar)

mina_gmm = GaussianMixture(n_components = 6, max_iter = 2000, covariance_type='diag',n_init = 3)
mina_gmm.fit(featuresMina)

mostafa_gmm = GaussianMixture(n_components = 6, max_iter = 2000, covariance_type='diag',n_init = 3)
mostafa_gmm.fit(featuresmostafa)


GaussianMixture(covariance_type='diag', max_iter=2000, n_components=6, n_init=3)

In [85]:
pickle.dump(mayar_gmm,open('./models/mayar.gmm','wb'))
pickle.dump(mina_gmm,open('./models/mina.gmm','wb'))
pickle.dump(magdy_gmm,open('./models/magdy.gmm','wb'))
pickle.dump(mostafa_gmm,open('./models/mostafa.gmm','wb'))

In [88]:
# test = extract_features('./static/assets/recordedAudio.wav')
test = extract_features('./files/test/recordedAudio17.wav')

[[-6.23181134  0.06484397  0.58918454 ...  0.38103895  0.55428167
  -0.49697659]
 [-0.1196325  -2.31669386  0.65715876 ...  1.31532183  1.53700142
   0.23591274]
 [ 0.14348047 -1.97116466  0.69343498 ...  1.21424472  1.18825169
  -0.8700939 ]
 ...
 [ 0.45006605 -1.46346407 -2.13764065 ...  1.05199199 -1.16805713
  -0.54094215]
 [ 0.36448229 -0.92188896 -1.27988075 ...  1.43265068 -0.97487533
  -0.50519503]
 [ 0.22815549 -1.40567788 -1.91131402 ...  1.08598233 -1.54483275
  -1.32679634]]
297
20


In [89]:
scores_1=np.array(magdy_gmm.score(test))
scores_2=np.array(mayar_gmm.score(test))
scores_3=np.array(mina_gmm.score(test))
scores_4=np.array(mostafa_gmm.score(test))


print(scores_1)
print(scores_2)
print(scores_3)
print(scores_4)





-27.713745902190983
-32.99565756419159
-30.153814692795482
-33.146425600324996
